In [ ]:
import numpy
import math

Let's define a function that operates on two inputs

In [ ]:
def trig(a, b):
    return math.sin(a**2) * math.exp(b)

In [ ]:
trig(1, 1)

Seems reasonable.  However, the `math` library only works on scalars.  If we try to pass in arrays, we'll get an error.

In [ ]:
a = numpy.ones((5,5))
b = numpy.ones((5,5))

In [ ]:
trig(a, b)

In [ ]:
from numba import vectorize

In [ ]:
vec_trig = vectorize()(trig)

In [ ]:
vec_trig(a, b)

And just like that, the scalar function `trig` is now a NumPy `ufunc` called `vec_trig`

Note that this is a "Dynamic UFunc" with no signature given.  

How does it compare to just using NumPy?  Let's check

In [ ]:
def numpy_trig(a, b):
    return numpy.sin(a**2) * numpy.exp(b)

In [ ]:
a = numpy.random.random((1000, 1000))
b = numpy.random.random((1000, 1000))

In [ ]:
%%timeit
numpy_trig(a, b)

In [ ]:
%%timeit
vec_trig(a, b)

What happens if we do specify a signature?  Is there a speed boost?

In [ ]:
vec_trig = vectorize(["float64(float64, float64)"])(trig)

In [ ]:
%%timeit
vec_trig(a, b)

No, not really.  But(!), if we have a signature, then we can add the target `kwarg`.

In [ ]:
vec_trig = vectorize(["float64(float64, float64)"], target='parallel')(trig)

In [ ]:
%%timeit
vec_trig(a, b)

# Clipping an array

In [ ]:
def truncate(a, amin, amax):
    if a < amin:
        a = amin
    elif a > amax:
        a = amax
    return a

In [ ]:
vec_truncate_serial = vectorize(['float64(float64, float64, float64)'])(truncate)
vec_truncate_par = vectorize(['float64(float64, float64, float64)'], target='parallel')(truncate)

In [ ]:
a = numpy.random.random((5000))

In [ ]:
amin = .2
amax = .6

In [ ]:
%timeit vec_truncate_serial(a, amin, amax)

In [ ]:
%timeit vec_truncate_par(a, amin, amax)

In [ ]:
%timeit numpy.clip(a, amin, amax)

In [ ]:
a = numpy.random.random((50000))

In [ ]:
%timeit vec_truncate_serial(a, amin, amax)

In [ ]:
%timeit vec_truncate_par(a, amin, amax)

In [ ]:
%timeit numpy.clip(a, amin, amax)